<a href="https://colab.research.google.com/github/deeptea22/tcsinframind/blob/main/tcsinframind/Image%20Captioning%20Model/exampleImageCaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# linear algebra 
import numpy as np 
# data processing, CSV file I / O (e.g. pd.read_csv) 
import pandas as pd 
import os 
import tensorflow as tf 
from keras.preprocessing.sequence import pad_sequences 
from keras.preprocessing.text import Tokenizer 
from keras.models import Model 
from keras.layers import Flatten, Dense, LSTM, Dropout, Embedding, Activation 
from keras.layers import concatenate, BatchNormalization, Input
from keras.layers.merge import add 
from keras.utils import to_categorical, plot_model 
from keras.applications.inception_v3 import InceptionV3, preprocess_input 
import matplotlib.pyplot as plt # for plotting data 
import cv2 
import string

In [ ]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fashiontest.txt to fashiontest.txt
Saving fashiontoken.txt to fashiontoken.txt
Saving fashiontrain.txt to fashiontrain.txt
Saving imagelist.txt to imagelist.txt


In [ ]:
import zipfile 
!unzip /content/drive/MyDrive/CompImages.zip
!unzip /content/drive/MyDrive/glove200d.zip

Archive:  /content/drive/MyDrive/CompImages.zip
  inflating: Images/10054.jpg        
  inflating: Images/10062.jpg        
  inflating: Images/10065.jpg        
  inflating: Images/10096.jpg        
  inflating: Images/10250.jpg        
  inflating: Images/10257.jpg        
  inflating: Images/10259.jpg        
  inflating: Images/10266.jpg        
  inflating: Images/10268.jpg        
  inflating: Images/10292.jpg        
  inflating: Images/10401.jpg        
  inflating: Images/10406.jpg        
  inflating: Images/10408.jpg        
  inflating: Images/10430.jpg        
  inflating: Images/10439.jpg        
  inflating: Images/10602.jpg        
  inflating: Images/10633.jpg        
  inflating: Images/10634.jpg        
  inflating: Images/10850.jpg        
  inflating: Images/10857.jpg        
  inflating: Images/10859.jpg        
  inflating: Images/10861.jpg        
  inflating: Images/10866.jpg        
  inflating: Images/11143.jpg        
  inflating: Images/11144.jpg        
  

In [ ]:
token_path = 'fashiontoken.txt'
doc = open(token_path, 'r', encoding = 'utf-8').read() 
descriptions = dict()
for line in doc.split('\n'):
        tokens = line.split()
        if len(line) > 2:
          image_id = tokens[0].split('.')[0]
          image_desc = ' '.join(tokens[1:])
          if image_id not in descriptions:
              descriptions[image_id] = list()
          descriptions[image_id].append(image_desc)

In [ ]:
table = str.maketrans('', '', string.punctuation)
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc]
        desc = [w.translate(table) for w in desc]
        desc_list[i] =  ' '.join(desc)

In [ ]:
#pic = 'crop-top-1.jpg'
images_path = 'Images/'
#x=plt.imread(images_path+pic)
#plt.imshow(x)
#plt.show()
#descriptions['crop-top-1']

In [ ]:
vocabulary = set()
for key in descriptions.keys():
        [vocabulary.update(d.split()) for d in descriptions[key]]
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 1127


In [ ]:
lines = list()
for key, desc_list in descriptions.items():
    for desc in desc_list:
        lines.append(key + ' ' + desc)
new_descriptions = '\n'.join(lines)

In [ ]:
train_images_path = 'fashiontrain.txt'
doc = open(train_images_path,'r').read()
dataset = list()
for line in doc.split('\n'):
    if len(line) > 1:
      identifier = line.split('.')[0]
      dataset.append(identifier)

train = set(dataset)

In [ ]:
import glob
test_images_path = 'fashiontest.txt'
img = glob.glob(images_path + '*.jpg')
train_images = set(open(train_images_path, 'r').read().strip().split('\n'))
train_img = []
for i in img: 
    if i[len(images_path):] in train_images:
        train_img.append(i)

test_images = set(open(test_images_path, 'r').read().strip().split('\n'))
test_img = []
for i in img: 
    if i[len(images_path):] in test_images: 
        test_img.append(i)

In [ ]:
train_descriptions = dict()
for line in new_descriptions.split('\n'):
    tokens = line.split()
    image_id, image_desc = tokens[0], tokens[1:]
    if image_id in train:
        if image_id not in train_descriptions:
            train_descriptions[image_id] = list()
        desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
        train_descriptions[image_id].append(desc)

In [ ]:
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)

In [ ]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))

Vocabulary = 444


In [ ]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

In [ ]:
all_desc = list()
for key in train_descriptions.keys():
    [all_desc.append(d) for d in train_descriptions[key]]
lines = all_desc
max_length = max(len(d.split()) for d in lines)

print('Description Length: %d' % max_length)

Description Length: 13


In [ ]:
embeddings_index = {} 
glove_path ='glove.6B.200d.txt'
f = open(glove_path, 'r', encoding = 'utf-8').read() 
for line in f.split("\n"):
    values = line.split(" ")
   # print(values)
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

In [ ]:
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
model = InceptionV3(weights='imagenet')

96116736/96112376 [==============================] - 1s 0us/step


In [ ]:
model_new = Model(model.input, model.layers[-2].output)

In [ ]:
from keras.preprocessing import image
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
def encode(image):
    image = preprocess(image) 
    fea_vec = model_new.predict(image) 
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1])
    return fea_vec

encoding_train = {}
for img in train_img:
    encoding_train[img[len(images_path):]] = encode(img)
train_features = encoding_train

encoding_test = {}
for img in test_img:
    encoding_test[img[len(images_path):]] = encode(img)

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 13, 200)      89000       input_3[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
X1, X2, y = list(), list(), list() 
for key, des_list in train_descriptions.items(): 
    pic = train_features[key+".jpg"] 
    for cap in des_list: 
        seq = [wordtoix[word] for word in cap.split(' ') if word in wordtoix] 
        for i in range(1, len(seq)): 
            in_seq, out_seq = seq[:i], seq[i] 
            in_seq = pad_sequences([in_seq], maxlen = max_length)[0] 
            out_seq = to_categorical([out_seq], num_classes = vocab_size)[0] 
            # store 
            X1.append(pic) 
            X2.append(in_seq) 
            y.append(out_seq) 
  
X2 = np.array(X2) 
X1 = np.array(X1) 
y = np.array(y)

In [ ]:
model.fit([X1, X2], y, epochs = 10, batch_size = 256) 

Epoch 1/10
97/97 [==============================] - 31s 270ms/step - loss: 4.4577
Epoch 2/10
97/97 [==============================] - 26s 270ms/step - loss: 2.3341
Epoch 3/10
97/97 [==============================] - 27s 282ms/step - loss: 1.5485
Epoch 4/10
97/97 [==============================] - 26s 272ms/step - loss: 1.1226
Epoch 5/10
97/97 [==============================] - 26s 272ms/step - loss: 0.8150
Epoch 6/10
97/97 [==============================] - 26s 272ms/step - loss: 0.6024
Epoch 7/10
97/97 [==============================] - 26s 267ms/step - loss: 0.4626
Epoch 8/10
97/97 [==============================] - 26s 265ms/step - loss: 0.3679
Epoch 9/10
97/97 [==============================] - 26s 268ms/step - loss: 0.2943
Epoch 10/10
97/97 [==============================] - 26s 266ms/step - loss: 0.2291


In [ ]:
def greedy_search(pic): 
    start = 'startseq'
    for i in range(max_length): 
        seq = [wordtoix[word] for word in start.split() if word in wordtoix] 
        seq = pad_sequences([seq], maxlen = max_length) 
        yhat = model.predict([pic, seq]) 
        yhat = np.argmax(yhat) 
        word = ixtoword[yhat] 
        start += ' ' + word 
        if word == 'endseq': 
            break
    final = start.split() 
    final = final[1:-1] 
    final = ' '.join(final) 
    return final

In [ ]:
path = 'imagelist.txt'
doc = open(path,'r').read()
for line in doc.split(','):
  pic = line[1:len(line)-1]
  encoding_test.update(encoding_train)
  image = encoding_test[pic].reshape((1,2048))
  x=plt.imread(images_path+pic)
  #plt.imshow(x)
  #plt.show()
  with open('jandesc.txt', 'a') as f:
      f.write(line[1:len(line)-1])
      f.write(":")
      f.write(greedy_search(image))
      f.write(",")
  #print("Greedy Search:",greedy_search(image))
files.download('jandesc.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>